In [3]:
import json, re, time
import ollama

#____ВОПРОС____
question_test = "Стоит ли использовать искусственный интеллект, если с его помощью можно автоматизировать многие процессы, но он не всегда бывает стабилен?"


def binary_answer(question, model_name="llama3.2", temperature=0.2, top_p=0.9, seed=8, num_predict=200):

    #____первичная оценка вопроса ollama____
    prompt_A = (
        "Сохранив суть вопроса, преобразуй его в более чёткий вопрос - такой, на который можно ответить да/нет и корректно выдели отдельно допущения и критерии вопроса.\n"
        "Верни строго один JSON-объект без текста до/после, поля:\n"
        "{ \"proposition\": <строка>, \"assumptions\": [строки], \"criteria\": [строки] }.\n"
        "Вопрос:\n\n"
        + question
    )
    resp_A = ollama.generate(
        model=model_name,
        prompt=prompt_A,
        options={"temperature": temperature, "top_p": top_p, "seed": seed, "num_predict": num_predict}
    )
    text_A = resp_A["response"].strip()
    
    # распарсить JSON; при неудаче — более строгий повтор
    ok_A = False # флаг успешного парсинга
    attempt_a = 0  # счётчик попыток
    while not ok_A and attempt_a < 3:
        attempt_a += 1
        try:
            data_A = json.loads(text_A)
            ok_A = True
        except Exception as e:
            print("Не JSON, более строгий повтор:...")
            prompt_A_strict = (
                "СТРОГО верни валидный JSON без лишних символов. Формат:\n"
                "{ \"proposition\": <строка>, \"assumptions\": [строки], \"criteria\": [строки] }.\n\nВопрос:\n" + text_A
            )
            resp_A = ollama.generate(
                model=model_name,
                prompt=prompt_A_strict,
                options={"temperature": 0.0, "top_p": 1.0, "seed": seed+attempt_a, "num_predict": 200}
            ) # seed: seed+attempt - изменённый сид для разнообразия
            text_A = resp_A["response"].strip()
    # если не получен успешный ответ, используется типизированная заготовка
    if not ok_A:
        data_A = {"proposition": question, "assumptions": ["Наиболее упрощённое рассуждение."], "criteria": ["Базовый ответ."]}
    
    #____анализ пропозиции по результатам оценки ollama____
    prop, assm, crit = data_A.get("proposition", question), data_A.get("assumptions", []), data_A.get("criteria", [])
    context_block = "\n".join(filter(None, [
        f"Пропозиция: {prop}",
        f"Допущения: {'; '.join(assm)}" if assm else "",
        f"Критерии: {'; '.join(crit)}" if crit else ""
    ]))
    prompt_B = (
        "Проанализируй пропозицию да/нет кратко по-русски на 2–4 предложения, ссылаясь на критерии/допущения, "
        "и завершай - VERDICT: YES или VERDICT: NO. Никаких других маркеров после вердикта.\n\n"
        + context_block
    )
    resp_B = ollama.generate(
        model=model_name,
        prompt=prompt_B,
        options={"temperature": 0.2, "top_p": top_p, "seed": seed, "num_predict": 240}
    )
    text_B = resp_B["response"].strip()
    
    #____приведение вердикта к бинарному формату____
    binary = None
    verdict = None
    # поиск VERDICT: YES/NO
    lit = re.search(r"VERDICT:\s*(YES|NO)\b", text_B, flags=re.IGNORECASE)
    if lit:
        verdict = lit.group(1).upper()
    
    # при неудаче повторный запрос на yes/no
    attempt_b = 0
    while verdict is None and attempt_b < 3:
        attempt_b += 1
        resp = ollama.generate(
            model=model_name,
            prompt="Ответь строго одним словом на английском: YES или NO. Только это слово. Пропозиция:\n\n" + prop,
            options={"temperature": 0.0, "top_p": 1.0, "seed": seed+attempt_b, "num_predict": 10}
        )
        txt = resp["response"].strip()
        lit = re.search(r"\b(YES|NO)\b", txt, flags=re.IGNORECASE)
        if lit:
            verdict = lit.group(1).upper()
    
    # YES/NO в 1/0
    if verdict == "YES":
        binary = 1
    elif verdict == "NO":
        binary = 0
    
    # при неудаче повторный запрос на 1/0
    attempt_c = 0
    while binary is None and attempt_c < 3:
        attempt_c += 1
        rule = (
            "Выбери 1 (за) или 0 (против) по пропозиции выше. Если аргументы равны — выбери 0. "
            "Верни строго одну цифру без текста."
        )
        resp = ollama.generate(
            model=model_name,
            prompt=rule + "\n\nПропозиция: " + prop,
            options={"temperature": 0.0, "top_p": 1.0, "seed": seed+10+attempt_c, "num_predict": 5}
        )
        txt = resp["response"].strip()
        lit_2 = re.search(r"^(0|1)$", txt)
        if lit_2:
            binary = int(lit_2.group(1))
    # при неудаче - поиск по всем ответам
    if binary is None:
        # YES/NO/ДА/НЕТ из всех ответов
        text_all = (text_A + "\n" + text_B).lower()
        if re.search(r"\b(yes|да)\b", text_all): binary = 1
        if re.search(r"\b(no|нет)\b", text_all): binary = 0 if binary is None else binary
    # при неудаче - последняя попытка строгой оценки моделью
    if binary is None:
        resp = ollama.generate(
            model=model_name,
            prompt="Верни строго одну цифру: 1 (за) или 0 (против) по пропозиции ниже. Без комментариев.\n\n" + prop,
            options={"temperature": 0.0, "top_p": 1.0, "seed": seed+99, "num_predict": 5}
        )
        txt = resp["response"].strip()
        lit_3 = re.search(r"^(0|1)$", txt)
        if lit_3:
            binary = int(lit_3.group(1))
    # при неудаче - дефолтное значение 0
    if binary is None:
        binary = 0
        
    return binary


#____ТЕСТ____
if __name__ == "__main__":
    binary_test = binary_answer(question_test)
    print(binary_test)

1
